#**Task-1:** <br>Create An Automated Pipeline For Data Ingestion, Cleaning, Transformation, And Loading.##

In [ ]:
import pandas as pd
path = '/content/Titanic-Dataset.csv'
df = pd.read_csv(path)
print(df.head())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [ ]:
#Import necessary libraries
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import joblib

### [1] Create An Automated Pipeline For Data Ingestion, Cleaning.

(A) Data Ingestion

In [ ]:
#Data Ingestion--> Load dataset from CSV
def load_data(path):
    return pd.read_csv(path)
df = load_data(path)
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


(B) Data Cleaning

In [ ]:
#Data Cleaning--> Droping irrelevant columns and handle missing values
def clean_data(df):
    df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    return df
cleaned_df = clean_data(df)
print(cleaned_df.head())

   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S


### [2] Create An Automated Pipeline For Transformation, And Loading.

(A) Data Transformation<br> Steps:<br> 1) Separate Numerical and categorical data<br> 2)Define numerical pipeline<br>3)Define categorical pipiline<br>4)Combine bith num and cat pipelines<br>5)Apply Transformation

In [ ]:
def transform_data(df):

# Separate target variable (assuming 'Survived' is the target)
    X_features = df.drop('Survived', axis=1)
    y = df['Survived']

#------------Step-1------------------------
#Separate numerical and categorical columns

    num_cols = X_features.select_dtypes(include=['int64', 'float64']).columns.tolist()
    cat_cols = X_features.select_dtypes(include=['object']).columns.tolist()
    print("Numerical columns:", num_cols)
    print("Categorical columns:", cat_cols)

#------------Step 2-------------------------
#Define numerical pipeline
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

#------------Step 3----------------------
#Define categorical pipeline
    cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

#------------Step 4----------------------
#Combine both pipelines
    full_pipeline = ColumnTransformer([
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ])

#------------Step 5-----------------------
#Apply transformations
    X_prepared = full_pipeline.fit_transform(X_features)
    print("Transformed shape:", X_prepared.shape)

# Display transformed DataFrame
    try:
        # Need to handle potential errors if the encoder is not present in the full_pipeline
        if 'cat' in full_pipeline.named_transformers_ and hasattr(full_pipeline.named_transformers_['cat'], 'named_steps') and 'encoder' in full_pipeline.named_transformers_['cat'].named_steps:
             cat_features = full_pipeline.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(cat_cols)
             all_columns = num_cols + cat_features.tolist()
             transformed_df = pd.DataFrame(X_prepared, columns=all_columns)
             print("Preview of transformed data:")
             display(transformed_df.head())
        else:
             print("Could not display transformed DataFrame: Categorical encoder not found in the pipeline.")

    except Exception as e:
        print(" Could not display transformed DataFrame:", e)

    return X_prepared, y, full_pipeline

# Transform the data (Now calling the function after it's defined)
# Use the cleaned_df for transformation
X_prepared, y, pipeline = transform_data(cleaned_df)

print("\nX_prepared shape:", X_prepared.shape)
print("y shape:", y.shape)
print("Pipeline:", pipeline)

Numerical columns: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Categorical columns: ['Sex', 'Embarked']
Transformed shape: (891, 10)
Preview of transformed data:


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,0.0,1.0,0.0,0.0,1.0
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,1.0,0.0,1.0,0.0,0.0
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,1.0,0.0,0.0,0.0,1.0
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,1.0,0.0,0.0,0.0,1.0
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,0.0,1.0,0.0,0.0,1.0



X_prepared shape: (891, 10)
y shape: (891,)
Pipeline: ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Sex', 'Embarked'])])


(B) Data Loading<br> Steps:<br> 1)Splitting the data <br>2)Saving the pipelines

In [ ]:
#------------------Step:1--------------------
#Splitting the Data
def split_data(X, y):
    """Split into training and testing sets"""
    from sklearn.model_selection import train_test_split
    print(" Splitting data into train and test sets...")
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Call the function
X_train, X_test, y_train, y_test = split_data(X_prepared, y)
print(" Training data shape:", X_train.shape)
print(" Testing data shape:", X_test.shape)


#------------------Step:2--------------------
#Saving the pipeline
def save_pipeline(pipeline, filename='titanic_pipeline.pkl'):
    """Save the transformation pipeline"""
    import joblib
    joblib.dump(pipeline, filename)
    print(f" Pipeline saved to '{filename}'")

# Call the function
save_pipeline(pipeline)




 Splitting data into train and test sets...
 Training data shape: (712, 10)
 Testing data shape: (179, 10)
 Pipeline saved to 'titanic_pipeline.pkl'


### **Overall ETL(Extract-> Transform->Load) Process**

In [ ]:
def run_etl(data_path='/content/Titanic-Dataset.csv'):
    print("Loading data...")
    df = load_data(data_path)

    print("Cleaning data...")
    df = clean_data(df)

    print("[Transforming data...")
    X, y, pipeline = transform_data(df)

    print("Splitting data...")
    X_train, X_test, y_train, y_test = split_data(X, y)

    print("Saving pipeline...")
    save_pipeline(pipeline)

    print(" ETL process completed.")
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = run_etl('/content/Titanic-Dataset.csv')

print("\nTraining features shape:", X_train.shape)
print("Training labels shape:", y_train.shape)

Loading data...
Cleaning data...
[Transforming data...
Numerical columns: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Categorical columns: ['Sex', 'Embarked']
Transformed shape: (891, 10)
Preview of transformed data:


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.827377,-0.592481,0.432793,-0.473674,-0.502445,0.0,1.0,0.0,0.0,1.0
1,-1.566107,0.638789,0.432793,-0.473674,0.786845,1.0,0.0,1.0,0.0,0.0
2,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,1.0,0.0,0.0,0.0,1.0
3,-1.566107,0.407926,0.432793,-0.473674,0.420730,1.0,0.0,0.0,0.0,1.0
4,0.827377,0.407926,-0.474545,-0.473674,-0.486337,0.0,1.0,0.0,0.0,1.0


Splitting data...
 Splitting data into train and test sets...
Saving pipeline...
 Pipeline saved to 'titanic_pipeline.pkl'
 ETL process completed.

Training features shape: (712, 10)
Training labels shape: (712,)
